In [2]:
import pandas as pd
import tabula
import numpy as np

In [3]:
lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

vecteur = [lieux[0] if i < 4 else
           lieux[1] if i < 8 else
           lieux[2] if i < 13 else
           lieux[3] if i < 15 else
           lieux[4] if i < 17 else
           lieux[5] if i < 19 else
           lieux[6] if i < 29 else
           lieux[7] if i < 35 else
           lieux[8] if i < 48 else
           lieux[9] if i < 58 else
           lieux[10] if i < 63 else
           lieux[11] if i < 68 else
           lieux[12] if i < 74 else
           lieux[13] if i < 78 else
           lieux[14] if i < 79 else
           lieux[15] if i < 83 else
           'Valeur par défaut' for i in range(83)]


# Lien vers le PDF contenant le tableau
lien_pdf = "https://medias.paris2024.org/uploads/2022/07/Calendrier-par-epreuves-des-Jeux-Olympiques-de-Paris-2024.pdf"

try : 
    # Utilisation de tabula pour extraire le tableau du PDF
    # Utilisation de l'option 'pages' pour spécifier la page ou les pages contenant le tableau
    df_list = tabula.read_pdf(lien_pdf, pages=[1, 2], stream=True)  # Adapter les options selon vos besoins

    # Concaténer les DataFrames obtenus des différentes pages en un seul DataFrame
    df = pd.concat(df_list)

    # Supprimer les 28 dernières lignes
    df = df.iloc[:-32]
    df.drop(24, inplace=True)
    df.drop(23, inplace=True)
    df.drop(55, inplace=True)

    # Supprimer la première ligne du DataFrame
    df = df.drop(df.index[[0, 2]])  # Suppression de la première ligne
    df = df.drop(df.columns[[0]], axis=1)

    # Supprimer les lignes contenant uniquement des NaN
    df = df.dropna(how='all')
    
    df.columns = df.iloc[0]
    df.drop(1, inplace=True)

    df.index = vecteur
    
    df = df.replace('<', np.nan)

except Exception as e:
        print("Une erreur s'est produite :", str(e))




Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [4]:
def format_date_horaire(cell, index_value):
    if isinstance(cell, str) and '-' in cell:
        date_month = '07' if int(index_value) in range(24, 32) else '08'
        year = '2024'
        
        date = f"{index_value}/{date_month}/{year}"
        horaire1, horaire2 = cell.split('-')
        
        return f"{date} {horaire1.strip()} - {horaire2.strip()}"
    else:
        return cell

# Appliquer la fonction à chaque cellule du DataFrame
for column in df.columns:
    df[column] = df.apply(lambda x: format_date_horaire(x[column], column), axis=1)


In [5]:
# Supprimer les titres des index
df.index.name = None

# Ajouter Lieux pour la première colonne
df.reset_index(inplace=True)
df.columns = ['Lieux'] + list(df.columns[1:])  # Renommer la première colonne en 'Lieux'

In [6]:
# Créer un dictionnaire pour stocker les valeurs non vides associées à chaque lieu
data = {}
for idx, row in df.iterrows():
    lieu = row['Lieux']
    if lieu not in data:
        data[lieu] = []

    # Récupérer les valeurs non vides et les ajouter à la liste des valeurs du lieu
    values = row.drop('Lieux').dropna().tolist()
    data[lieu].extend(values)

# Créer un nouveau DataFrame à partir du dictionnaire
new_df = pd.DataFrame.from_dict(data, orient='index')

lieux_a_supprimer = ['arena-paris-nord', 'site-escalade-bourget', 'centre-aquatique']

# Suppression des lignes spécifiées
new_df = new_df.drop(lieux_a_supprimer, axis=0)

In [7]:
# Fonction pour convertir le format de date
def convert_date_format(entry):
    if isinstance(entry, str):
        parts = entry.split(' ')
        if len(parts) == 5:
            date = parts[0]
            start_time = parts[1]
            return f"{date} {start_time}"
    return entry

# Appliquer la fonction sur toutes les colonnes du DataFrame
for col in new_df.columns:
    new_df[col] = new_df[col].apply(convert_date_format)


In [8]:
# Appliquer une fonction pour supprimer la partie après le tiret dans chaque cellule
horaires_début = new_df.applymap(lambda x: x.split(' -')[0] if isinstance(x, str) else x)

# Appliquer une fonction pour supprimer les 4 caractères avant le tiret dans chaque cellule
horaires_fin = new_df.applymap(lambda x: pd.Series(x).replace(r'.{5}\s-\s', '', regex=True)[0] if isinstance(x, str) else x)

horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_début = horaires_début.dropna(axis=1, how='all')

horaires_fin = horaires_fin.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_fin = horaires_fin.dropna(axis=1, how='all')

# Afficher le DataFrame résultant
display(horaires_début)
display(horaires_fin)


/tmp/ipykernel_4737/1552186039.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  horaires_début = new_df.applymap(lambda x: x.split(' -')[0] if isinstance(x, str) else x)
/tmp/ipykernel_4737/1552186039.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  horaires_fin = new_df.applymap(lambda x: pd.Series(x).replace(r'.{5}\s-\s', '', regex=True)[0] if isinstance(x, str) else x)
/tmp/ipykernel_4737/1552186039.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
/tmp/ipykernel_4737/1552186039.py:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
/tmp/ipykernel_4737/1552186039.py:10: Futu

,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
stade-tour-eiffel,2024-07-28 09:00:00,2024-07-29 09:00:00,2024-07-30 09:00:00,2024-07-31 09:00:00,2024-01-08 09:00:00,2024-02-08 09:00:00,2024-03-08 09:00:00,2024-04-08 09:00:00,2024-05-08 09:00:00,2024-07-27 14:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
arena-champ-de-mars,2024-07-27 10:00:00,2024-07-28 10:00:00,2024-07-29 10:00:00,2024-07-30 10:00:00,2024-07-31 10:00:00,2024-01-08 10:00:00,2024-02-08 10:00:00,2024-03-08 08:00:00,2024-07-27 16:00:00,2024-07-28 16:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
grand-palais,2024-07-27 10:00:00,2024-07-28 09:30:00,2024-07-29 09:30:00,2024-07-30 12:00:00,2024-07-31 12:00:00,2024-01-08 10:00:00,2024-02-08 12:00:00,2024-03-08 11:30:00,2024-04-08 10:00:00,2024-07-27 19:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
invalides,2024-07-25 09:30:00,2024-07-28 09:30:00,2024-07-29 09:30:00,2024-07-30 12:00:00,2024-07-31 12:00:00,2024-01-08 09:30:00,2024-02-08 09:30:00,2024-03-08 09:30:00,2024-04-08 09:30:00,2024-07-25 14:15:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
pont-alexandre-iii,2024-07-30 08:00:00,2024-07-31 08:00:00,2024-05-08 08:00:00,2024-08-08 07:30:00,2024-09-08 07:30:00,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
trocadero,2024-01-08 07:30:00,2024-03-08 11:00:00,2024-04-08 14:00:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
concorde,2024-01-08 09:00:00,2024-02-08 09:00:00,2024-01-08 12:30:00,2024-07-30 17:30:00,2024-07-31 17:30:00,2024-01-08 18:00:00,2024-02-08 17:30:00,2024-03-08 17:30:00,2024-04-08 17:30:00,2024-05-08 17:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
stade-roland-garros,2024-06-08 17:30:00,2024-07-08 17:30:00,2024-07-27 12:00:00,2024-07-28 12:00:00,2024-07-29 12:00:00,2024-07-30 12:00:00,2024-07-31 12:00:00,2024-01-08 12:00:00,2024-02-08 12:00:00,2024-03-08 12:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
arena-paris-sud,2024-06-08 21:30:00,2024-07-08 21:30:00,2024-08-08 21:30:00,2024-09-08 21:30:00,2024-10-08 21:30:00,2024-07-27 09:00:00,2024-07-28 09:00:00,2024-07-29 09:00:00,2024-07-30 09:00:00,2024-07-31 09:00:00,...,2024-10-08 11:30:00,2024-11-08 11:30:00,2024-07-08 15:00:00,2024-08-08 15:00:00,2024-09-08 15:00:00,2024-10-08 16:00:00,2024-07-08 19:30:00,2024-08-08 19:30:00,2024-09-08 19:30:00,2024-10-08 20:30:00
arena-bercy,2024-07-27 11:00:00,2024-07-28 09:30:00,2024-07-28 14:50:00,2024-05-08 12:00:00,2024-07-27 15:30:00,2024-07-28 18:00:00,2024-03-08 15:30:00,2024-04-08 15:00:00,2024-07-27 20:00:00,2024-07-28 21:10:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
stade-tour-eiffel,2024-07-28 13:00:00,2024-07-29 13:00:00,2024-07-30 13:00:00,2024-07-31 13:00:00,2024-01-08 13:00:00,2024-02-08 13:00:00,2024-03-08 13:00:00,2024-04-08 11:00:00,2024-05-08 11:00:00,2024-07-27 16:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
arena-champ-de-mars,2024-07-27 14:00:00,2024-07-28 14:00:00,2024-07-29 14:00:00,2024-07-30 14:00:00,2024-07-31 14:00:00,2024-01-08 14:00:00,2024-02-08 14:00:00,2024-03-08 14:00:00,2024-07-27 19:00:00,2024-07-28 19:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
grand-palais,2024-07-27 16:40:00,2024-07-28 16:50:00,2024-07-29 16:50:00,2024-07-30 17:30:00,2024-07-31 17:30:00,2024-01-08 17:10:00,2024-02-08 17:30:00,2024-03-08 17:00:00,2024-04-08 17:10:00,2024-07-27 22:50:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
invalides,2024-07-25 12:30:00,2024-07-28 11:05:00,2024-07-29 11:05:00,2024-07-30 15:55:00,2024-07-31 15:55:00,2024-01-08 13:25:00,2024-02-08 12:05:00,2024-03-08 11:15:00,2024-04-08 11:15:00,2024-07-25 17:15:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
pont-alexandre-iii,2024-07-30 10:30:00,2024-07-31 10:40:00,2024-05-08 10:10:00,2024-08-08 10:30:00,2024-09-08 10:30:00,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
trocadero,2024-01-08 11:45:00,2024-03-08 18:15:00,2024-04-08 18:45:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
concorde,2024-01-08 11:00:00,2024-02-08 11:00:00,2024-01-08 14:30:00,2024-07-30 19:30:00,2024-07-31 19:30:00,2024-01-08 20:00:00,2024-02-08 19:30:00,2024-03-08 19:30:00,2024-04-08 19:30:00,2024-05-08 19:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
stade-roland-garros,2024-06-08 19:00:00,2024-07-08 19:00:00,2024-07-27 17:00:00,2024-07-28 17:00:00,2024-07-29 17:00:00,2024-07-30 17:00:00,2024-07-31 17:00:00,2024-01-08 17:00:00,2024-02-08 17:00:00,2024-03-08 19:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
arena-paris-sud,2024-06-08 23:30:00,2024-07-08 23:30:00,2024-08-08 23:30:00,2024-09-08 23:30:00,2024-10-08 23:30:00,2024-07-27 11:30:00,2024-07-28 11:30:00,2024-07-29 11:30:00,2024-07-30 11:30:00,2024-07-31 11:30:00,...,2024-10-08 14:00:00,2024-11-08 14:00:00,2024-07-08 17:30:00,2024-08-08 17:30:00,2024-09-08 17:30:00,2024-10-08 18:30:00,2024-07-08 22:00:00,2024-08-08 22:00:00,2024-09-08 22:00:00,2024-10-08 23:00:00
arena-bercy,2024-07-27 13:30:00,2024-07-28 13:20:00,2024-07-28 16:30:00,2024-05-08 15:15:00,2024-07-27 18:00:00,2024-07-28 19:40:00,2024-03-08 18:00:00,2024-04-08 17:25:00,2024-07-27 22:30:00,2024-07-28 22:50:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [9]:
def adjust_time_moins_90min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Ajouter 1 heure et 30 minutes à l'objet datetime
        cell = cell - pd.Timedelta(hours=1, minutes=30)
    return cell

def adjust_time_plus_30min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Retirer 30 minutes à l'objet datetime
        cell = cell + pd.Timedelta(hours=0, minutes=30)
    return cell

# Parcourir chaque cellule du DataFrame et appliquer la fonction adjust_time
new_horaires_début_df = horaires_début.map(adjust_time_moins_90min)
new_horaires_fin_df = horaires_fin.map(adjust_time_plus_30min)

new_horaires_début_df.reset_index(inplace=True)
new_horaires_début_df.rename(columns={'index': 'Lieu'}, inplace=True)

new_horaires_fin_df.reset_index(inplace=True)
new_horaires_fin_df.rename(columns={'index': 'Lieu'}, inplace=True)

display(new_horaires_début_df)
display(new_horaires_fin_df)

new_horaires_fin_df.to_excel('new_horaires_fin_df.xlsx', index=False)
new_horaires_début_df.to_excel('new_horaires_début_df.xlsx', index=False)

,Lieu,0,1,2,3,4,5,6,7,8,...,112,113,114,115,116,117,118,119,120,121
0,stade-tour-eiffel,2024-07-28 07:30:00,2024-07-29 07:30:00,2024-07-30 07:30:00,2024-07-31 07:30:00,2024-01-08 07:30:00,2024-02-08 07:30:00,2024-03-08 07:30:00,2024-04-08 07:30:00,2024-05-08 07:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,arena-champ-de-mars,2024-07-27 08:30:00,2024-07-28 08:30:00,2024-07-29 08:30:00,2024-07-30 08:30:00,2024-07-31 08:30:00,2024-01-08 08:30:00,2024-02-08 08:30:00,2024-03-08 06:30:00,2024-07-27 14:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,grand-palais,2024-07-27 08:30:00,2024-07-28 08:00:00,2024-07-29 08:00:00,2024-07-30 10:30:00,2024-07-31 10:30:00,2024-01-08 08:30:00,2024-02-08 10:30:00,2024-03-08 10:00:00,2024-04-08 08:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,invalides,2024-07-25 08:00:00,2024-07-28 08:00:00,2024-07-29 08:00:00,2024-07-30 10:30:00,2024-07-31 10:30:00,2024-01-08 08:00:00,2024-02-08 08:00:00,2024-03-08 08:00:00,2024-04-08 08:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,pont-alexandre-iii,2024-07-30 06:30:00,2024-07-31 06:30:00,2024-05-08 06:30:00,2024-08-08 06:00:00,2024-09-08 06:00:00,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,trocadero,2024-01-08 06:00:00,2024-03-08 09:30:00,2024-04-08 12:30:00,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,concorde,2024-01-08 07:30:00,2024-02-08 07:30:00,2024-01-08 11:00:00,2024-07-30 16:00:00,2024-07-31 16:00:00,2024-01-08 16:30:00,2024-02-08 16:00:00,2024-03-08 16:00:00,2024-04-08 16:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,stade-roland-garros,2024-06-08 16:00:00,2024-07-08 16:00:00,2024-07-27 10:30:00,2024-07-28 10:30:00,2024-07-29 10:30:00,2024-07-30 10:30:00,2024-07-31 10:30:00,2024-01-08 10:30:00,2024-02-08 10:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,arena-paris-sud,2024-06-08 20:00:00,2024-07-08 20:00:00,2024-08-08 20:00:00,2024-09-08 20:00:00,2024-10-08 20:00:00,2024-07-27 07:30:00,2024-07-28 07:30:00,2024-07-29 07:30:00,2024-07-30 07:30:00,...,2024-10-08 10:00:00,2024-11-08 10:00:00,2024-07-08 13:30:00,2024-08-08 13:30:00,2024-09-08 13:30:00,2024-10-08 14:30:00,2024-07-08 18:00:00,2024-08-08 18:00:00,2024-09-08 18:00:00,2024-10-08 19:00:00
9,arena-bercy,2024-07-27 09:30:00,2024-07-28 08:00:00,2024-07-28 13:20:00,2024-05-08 10:30:00,2024-07-27 14:00:00,2024-07-28 16:30:00,2024-03-08 14:00:00,2024-04-08 13:30:00,2024-07-27 18:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


,Lieu,0,1,2,3,4,5,6,7,8,...,112,113,114,115,116,117,118,119,120,121
0,stade-tour-eiffel,2024-07-28 13:30:00,2024-07-29 13:30:00,2024-07-30 13:30:00,2024-07-31 13:30:00,2024-01-08 13:30:00,2024-02-08 13:30:00,2024-03-08 13:30:00,2024-04-08 11:30:00,2024-05-08 11:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,arena-champ-de-mars,2024-07-27 14:30:00,2024-07-28 14:30:00,2024-07-29 14:30:00,2024-07-30 14:30:00,2024-07-31 14:30:00,2024-01-08 14:30:00,2024-02-08 14:30:00,2024-03-08 14:30:00,2024-07-27 19:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,grand-palais,2024-07-27 17:10:00,2024-07-28 17:20:00,2024-07-29 17:20:00,2024-07-30 18:00:00,2024-07-31 18:00:00,2024-01-08 17:40:00,2024-02-08 18:00:00,2024-03-08 17:30:00,2024-04-08 17:40:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,invalides,2024-07-25 13:00:00,2024-07-28 11:35:00,2024-07-29 11:35:00,2024-07-30 16:25:00,2024-07-31 16:25:00,2024-01-08 13:55:00,2024-02-08 12:35:00,2024-03-08 11:45:00,2024-04-08 11:45:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,pont-alexandre-iii,2024-07-30 11:00:00,2024-07-31 11:10:00,2024-05-08 10:40:00,2024-08-08 11:00:00,2024-09-08 11:00:00,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,trocadero,2024-01-08 12:15:00,2024-03-08 18:45:00,2024-04-08 19:15:00,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,concorde,2024-01-08 11:30:00,2024-02-08 11:30:00,2024-01-08 15:00:00,2024-07-30 20:00:00,2024-07-31 20:00:00,2024-01-08 20:30:00,2024-02-08 20:00:00,2024-03-08 20:00:00,2024-04-08 20:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,stade-roland-garros,2024-06-08 19:30:00,2024-07-08 19:30:00,2024-07-27 17:30:00,2024-07-28 17:30:00,2024-07-29 17:30:00,2024-07-30 17:30:00,2024-07-31 17:30:00,2024-01-08 17:30:00,2024-02-08 17:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,arena-paris-sud,2024-06-09 00:00:00,2024-07-09 00:00:00,2024-08-09 00:00:00,2024-09-09 00:00:00,2024-10-09 00:00:00,2024-07-27 12:00:00,2024-07-28 12:00:00,2024-07-29 12:00:00,2024-07-30 12:00:00,...,2024-10-08 14:30:00,2024-11-08 14:30:00,2024-07-08 18:00:00,2024-08-08 18:00:00,2024-09-08 18:00:00,2024-10-08 19:00:00,2024-07-08 22:30:00,2024-08-08 22:30:00,2024-09-08 22:30:00,2024-10-08 23:30:00
9,arena-bercy,2024-07-27 14:00:00,2024-07-28 13:50:00,2024-07-28 17:00:00,2024-05-08 15:45:00,2024-07-27 18:30:00,2024-07-28 20:10:00,2024-03-08 18:30:00,2024-04-08 17:55:00,2024-07-27 23:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [10]:


def ajuster_plage_horaire(heure):
    if pd.notnull(heure):
        # Extraire la date et l'heure de la cellule
        date, time = str(heure).split(' ')
        heure, minute, seconde = map(int, time.split(':')[0:3])
        
        # Calculer la plage horaire correspondante
        plage_horaire = f"{heure:02d}:00:00 - {(heure+1)%24:02d}:00:00"
        
        # Construire la date avec la plage horaire
        return f"{date} {plage_horaire}"
    
    return heure

# Appliquer la fonction à toutes les colonnes (sauf la première)
new_horaires_début_df.iloc[:, 1:] = new_horaires_début_df.iloc[:, 1:].applymap(ajuster_plage_horaire)
new_horaires_fin_df.iloc[:, 1:] = new_horaires_début_df.iloc[:, 1:].applymap(ajuster_plage_horaire)


/tmp/ipykernel_4737/3235712404.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  plages_h_début.iloc[:, 1:] = new_horaires_début_df.iloc[:, 1:].applymap(ajuster_plage_horaire)
/tmp/ipykernel_4737/3235712404.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  plage_h_fin.iloc[:, 1:] = new_horaires_début_df.iloc[:, 1:].applymap(ajuster_plage_horaire)


ValueError: too many values to unpack (expected 2)

Ajout du script Nb de personnes + Arrêts + Lignes des lieux des événements

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# %% [markdown]
# Liste des lieux et url 

# %%

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_lines = {
            'Capacité': [],
            'Stations': []  # Création d'une liste pour stocker les stations
        }
        for paragraph in soup.find_all('p'):
            # Recherche des lignes contenant les Capacité spécifiques
            if ('Capacité' in paragraph.text or 
                'Capacité court Philippe-Chatrier' in paragraph.text or 
                'Volleyball' in paragraph.text or 
                'Pour la gymnastique rythmique' in paragraph.text):
                specific_lines['Capacité'].append(paragraph.text.strip())
                
            # Extraction des phrases après "desservie" ou "desservi"
            if 'desservie' in paragraph.text or 'desservi' in paragraph.text:
                sentences = paragraph.text.split('.')  # Séparation en phrases
                for sentence in sentences:
                    if 'desservie' in sentence or 'desservi' in sentence:
                        specific_lines['Stations'].append(sentence.strip())
        
        return specific_lines
    return None


# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Infos'])
# Expansion du dictionnaire pour créer les colonnes 'Capacité' et 'Stations'
df = pd.concat([df.drop(['Infos'], axis=1), df['Infos'].apply(pd.Series)], axis=1)


# %% [markdown]
# Compléter pour les données manquantes : Concorde 

# %%
# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[6, 1]=texte_places.strip()

# %% [markdown]
# Compléter pour les données manquantes : Arena Paris Sud

# %%
#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[8, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

# %%
def extraire_nombres(texte):
    nombres = re.findall(r'[-+]?\d*\.\d+|\d+', str(texte))  # Expression régulière pour les nombres entiers ou décimaux
    return ' '.join(nombres)  # Retourne les nombres extraits sous forme d'une chaîne séparée par des espaces

# Appliquer la fonction sur la colonne 'Capacité'
df['Nombre de personnes à l\'événément'] = df['Capacité'].apply(extraire_nombres)

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer "202" par une chaîne vide dans la colonne 'Nombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace('202', '')

# Supprimer les espaces dans la colonne 'NNombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace(' ', '')

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer les valeurs dans la colonne 'Nombres' aux lignes spécifiques
df.loc[2, 'Nombre de personnes à l\'événément'] = '8000'  # Remplacer la deuxième ligne par '8000'
df.loc[10, 'Nombre de personnes à l\'événément'] = '6700'  # Remplacer la 10ème ligne par '6700'
df.loc[14, 'Nombre de personnes à l\'événément'] = '6000'  # Remplacer la 14ème ligne par '6000'

# Fonction pour supprimer le zéro en tête des nombres
def remove_leading_zero(number):
    if number.startswith('0'):
        return number[1:]
    return number

# Appliquer la fonction à la colonne 'Nombres' pour supprimer les zéros en tête
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].astype(str).apply(remove_leading_zero)


# %%
# Boucle pour lire chaque ligne de la colonne 'Stations'
for index, station in df['Stations'].items():
    print(station)

# %%
arrêts=[['BIR HAKEIM', 'ALMA MARCEAU'],
        ['LA MOTTE-PICQUET-GRENELLE', 'LA MOTTE-PICQUET-GRENELLE', 'SEGUR'], 
        ['FRANKLIN D. ROOSEVELT', 'FRANKLIN D. ROOSEVELT'], 
        ['INVALIDES'], 
        ['INVALIDES'],
        ['IENA','TROCADERO'], 
        ['PALAIS-ROYAL','PALAIS-ROYAL','MADELEINE', 'MADELEINE', 'MADELEINE', 'OPERA'],
        ['PORTE DE SAINT-CLOUD','PORTE D\'AUTEUIL'], 
        ['PORTE DE VERSAILLES','BALARD','PORTE DE VANVES'], 
        ['BERCY','BERCY','GARE DE LYON'],
        ['PORTE DE LA CHAPELLE'],
        ['LA DEFENSE'],
        ['x'],
        ['SAINT-DENIS-PORTE DE PARIS'], 
        ['x'],
        ['x']]
 
df['Arrêts']=arrêts

# Fonction pour extraire les nombres après le mot "Métro"
def extract_numbers(text):
    matches = re.findall(r'Métro\s+(\d+)', str(text), flags=re.IGNORECASE)
    return matches

# Appliquer la fonction à la colonne 'Stations' pour créer une nouvelle colonne 'Nombres après Métro'
df['Lignes'] = df['Stations'].apply(extract_numbers)

# Compléter pour Franklin D
df.at[2, 'Lignes'] = [6, 9]

# %%
# Supprimer les colonnes 'Capacité' et 'Stations'
colonnes_a_supprimer = ['Capacité', 'Stations']
df = df.drop(colonnes_a_supprimer, axis=1)

# Calculer le nombre d'éléments dans chaque vecteur de la ligne 'Lignes'
df['Nombre de lignes'] = df['Lignes'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Convertir la colonne 'Nombre de personnes à l\'événément' en type numérique (int ou float)
df['Nombre de personnes à l\'événément'] = pd.to_numeric(df['Nombre de personnes à l\'événément'], errors='coerce')

# Créer la nouvelle colonne en calculant la division
df['Nombre de personnes par ligne'] = df['Nombre de personnes à l\'événément'] / df['Nombre de lignes']

# Définir l'option d'affichage pour éviter la notation scientifique
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# %%
# Identifier les indices des lignes contenant les valeurs spécifiques dans la colonne 'lieu'
indices_a_supprimer = df[df['Lieu'].isin(['centre-aquatique', 'site-escalade-bourget', 'arena-paris-nord'])].index

# Supprimer les lignes correspondantes
df = df.drop(indices_a_supprimer)

display(df)

df.to_excel('df.xlsx', index=False)

['Le Stade Tour Eiffel sera desservi par les gares «\xa0Champ de Mars Tour Eiffel\xa0» et «\xa0Pont de l’Alma\xa0» (RER C), par la station «\xa0Bir Hakeim\xa0» (Métro 6), et par la station «\xa0Alma Marceau\xa0» (Métro 9)']
['L’Arena Champ de Mars sera desservie par la gare «\xa0Champ de Mars Tour Eiffel\xa0» (RER C), la station «\xa0La Motte Piquet Grenelle\xa0» (Métro 6, Métro 8), et la station «\xa0Ségur\xa0» (Métro 10)']
['Le Grand Palais sera desservi par la station «\xa0Franklin D']
['Le site des Invalides sera desservi par la gare «\xa0Invalides\xa0» (RER C, Métro 8, Métro 13)']
['Le site du Pont Alexandre III sera desservi par la gare «\xa0Invalides\xa0» (RER C, Métro 8, Métro 13)']
['Le site du Trocadéro sera desservi par la station «\xa0Iéna\xa0» (Métro 9), la station Trocadéro (Métro 6 et 9) et la gare «\xa0Pont de l’Alma\xa0» (RER C)']
['Le site de La Concorde sera desservi par les stations «\xa0Palais Royal – Musée du Louvre\xa0» (Métro 1, Métro 7), «\xa0Madeleine\xa0» (Mé

/tmp/ipykernel_1465/3619698837.py:89: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texte_places = soup.find(text=lambda text: text and "places" in text)
/tmp/ipykernel_1465/3619698837.py:110: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)


,Lieu,Nombre de personnes à l'événément,Arrêts,Lignes,Nombre de lignes,Nombre de personnes par ligne
0,stade-tour-eiffel,12860,"[BIR HAKEIM, ALMA MARCEAU]","[6, 9]",2,6430.00
1,arena-champ-de-mars,8356,"[LA MOTTE-PICQUET-GRENELLE, LA MOTTE-PICQUET-G...","[6, 8, 10]",3,2785.33
2,grand-palais,8000,"[FRANKLIN D. ROOSEVELT, FRANKLIN D. ROOSEVELT]","[6, 9]",2,4000.00
3,invalides,8000,[INVALIDES],"[8, 13]",2,4000.00
4,pont-alexandre-iii,1000,[INVALIDES],"[8, 13]",2,500.00
5,trocadero,3349,"[IENA, TROCADERO]","[9, 6]",2,1674.50
6,concorde,35000,"[PALAIS-ROYAL, PALAIS-ROYAL, MADELEINE, MADELE...","[1, 7, 8, 12, 14, 3]",6,5833.33
7,stade-roland-garros,14962,"[PORTE DE SAINT-CLOUD, PORTE D'AUTEUIL]","[9, 10]",2,7481.00
8,arena-paris-sud,9000,"[PORTE DE VERSAILLES, BALARD, PORTE DE VANVES]","[12, 8, 13]",3,3000.00
9,arena-bercy,15000,"[BERCY, BERCY, GARE DE LYON]","[6, 14, 1]",3,5000.00


Copie du programme fréquentation_metro

In [ ]:
# Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

# %%
import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# %%
# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# %%
# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# %%
# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# %%
import re

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)

# %% [markdown]
# Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne

# %%
import pandas as pd
import numpy as np

tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print(tableau_fréquentation)

# %%
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
display(tableau_fréquentation)

# %%
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Afficher le DataFrame après suppression des colonnes
print(tableau_fréquentation)

# %%
# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

# Afficher le DataFrame après suppression des lettres spécifiées
print(tableau_fréquentation)

# %%
# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Remplacer "nan" par "NaN" dans les colonnes spécifiées
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].replace('nan', 'NaN')

# Afficher le DataFrame avec les remplacements effectués
print(tableau_fréquentation)

# %%
# Liste des colonnes à analyser
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Calcul du nombre de cases non vides pour chaque ligne
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].count(axis=1)

# Afficher le DataFrame avec la nouvelle colonne "nb_correspondance"
print(tableau_fréquentation)

# %%
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']

# Afficher le DataFrame avec les valeurs mises à jour
print(tableau_fréquentation)

# %%
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(somme_par_jour_arret)
display(moyenne_par_arret)

# %%
import pandas as pd

# Vos données et numéro
# Assumons que vous avez déjà vos données dans un DataFrame nommé tableau_fréquentation
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in x.values, axis=1)]
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Trafic_divisé'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

# %% [markdown]
# Compléter avec les valeurs manquantes

# %%
import difflib

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.6)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table



# %%

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Colonnes à supprimer
    columns_to_drop = ['Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé']

    # Vérifier si les colonnes existent avant de les supprimer
    columns_exist = [col for col in columns_to_drop if col in current_table.columns]

    # Supprimer les colonnes existantes
    if columns_exist:
        current_table = current_table.drop(columns=columns_exist)

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

    # Afficher le tableau mis à jour
    print(f"Tableau 'ligne numéro[{k_str}]' sans les colonnes 'Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé' :")
    display(tableaux[f'ligne numéro[{k_str}]'])


# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table
        # Afficher le tableau mis à jour
        print(f"Tableau 'ligne numéro[{k_str}]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :")
        display(tableaux[f'ligne numéro[{k_str}]'])

/tmp/ipykernel_1465/4252363584.py:7: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


       LIBELLE_ARRET  00:00-01:00  01:00-02:00  02:00-03:00  03:00-04:00  \
0           abbesses        10.18         3.20         0.02         0.00   
1              ablon         0.26         0.03         0.00         0.02   
2      acheres grand         0.78          NaN          NaN          NaN   
3      acheres ville         0.76         0.31         0.13         0.09   
4      aeroport cdg1         0.76         0.14         0.01         0.04   
..               ...          ...          ...          ...          ...   
751           vosves          NaN          NaN          NaN          NaN   
752  vulaines sur se         0.11          NaN          NaN          NaN   
753           wagram         3.63         1.43         0.10         0.00   
754           yerres         0.73         0.11         0.01         0.05   
755   yvris noisy gd         0.20         0.04          NaN          NaN   

     04:00-05:00  05:00-06:00  06:00-07:00  07:00-08:00  08:00-09:00  ...  \
0         

,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,7,Métro,REPUBLIQUE,11079708,3,5,8,9.00,11.00,Paris,11.00
1,9,Métro,LA DEFENSE,9256802,1,NaN,NaN,NaN,NaN,Puteaux,NaN
2,13,Métro,HOTEL DE VILLE,7251729,1,11,NaN,NaN,NaN,Paris,4.00
3,30,Métro,VILLEJUIF-LOUIS ARAGON,5218070,7,NaN,NaN,NaN,NaN,Villejuif,NaN
4,40,Métro,PALAIS-ROYAL,4822599,1,7,NaN,NaN,NaN,Paris,1.00
...,...,...,...,...,...,...,...,...,...,...,...
357,286,Métro,PICPUS,931602,6,NaN,NaN,NaN,NaN,Paris,12.00
358,287,Métro,TEMPLE,891858,3,NaN,NaN,NaN,NaN,Paris,2.00
359,289,Métro,QUAI DE LA RAPEE,798728,5,NaN,NaN,NaN,NaN,Paris,12.00
360,299,Métro,DANUBE,400157,7bis,NaN,NaN,NaN,NaN,Paris,19.00


     Rang                 Station    Trafic Correspondance_1 Correspondance_2  \
0       7              REPUBLIQUE  11079708                3                5   
1       9              LA DEFENSE   9256802                1              NaN   
2      13          HOTEL DE VILLE   7251729                1               11   
3      30  VILLEJUIF-LOUIS ARAGON   5218070                7              NaN   
4      40            PALAIS-ROYAL   4822599                1                7   
..    ...                     ...       ...              ...              ...   
357   286                  PICPUS    931602                6              NaN   
358   287                  TEMPLE    891858                3              NaN   
359   289        QUAI DE LA RAPEE    798728                5              NaN   
360   299                  DANUBE    400157             7bis              NaN   
361   303       PRE-SAINT-GERVAIS    282626             7bis              NaN   

    Correspondance_3  Corre

,JOUR,LIBELLE_ARRET,NB_VALD
0,2022-07-24,ABBESSES,1867
1,2022-07-24,ABLON,600
2,2022-07-24,ACHERES GRAND,39
3,2022-07-24,ACHERES VILLE,3002
4,2022-07-24,AEROPORT CDG1,2955
...,...,...,...
13776,2022-08-11,VOSVES,20
13777,2022-08-11,VULAINES SUR SE,44
13778,2022-08-11,WAGRAM,3063
13779,2022-08-11,YERRES,2729


...

,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,86,concorde,5,9636.53,10.30,2.87,0.08,0.04,0.02,2.15,...,66.58,69.70,54.94,27.15,19.30,18.36,17.88,1927.31,963.65,104976.33
1,105,pasteur,5,6307.74,7.42,2.12,0.13,0.00,0.00,2.47,...,43.50,44.12,35.49,20.42,12.83,11.10,10.56,1261.55,630.77,104976.33
2,233,abbesses,5,2277.32,10.18,3.20,0.02,0.00,NaN,1.31,...,45.59,43.24,37.00,23.36,19.02,19.36,18.35,455.46,227.73,104976.33
3,280,trinite-d'estienne d'orves,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104976.33
4,291,saint-georges,5,1449.11,9.83,3.71,0.10,0.00,0.01,2.00,...,47.79,42.88,34.41,21.11,16.17,26.32,17.00,289.82,144.91,104976.33
5,296,assemblee nationale,5,1602.26,14.18,8.04,0.10,0.01,0.00,3.03,...,65.30,64.65,41.95,25.79,16.80,15.93,18.23,320.45,160.23,104976.33
6,2,saint-lazare,5,127163.37,8.62,0.10,0.01,0.01,0.12,3.15,...,57.36,63.81,51.34,34.47,21.65,16.09,13.86,25432.67,12716.34,104976.33
7,2,saint-lazare,5,127163.37,8.06,1.80,0.14,0.04,0.08,8.17,...,85.45,92.97,77.22,51.26,27.60,21.59,14.81,25432.67,12716.34,104976.33
8,55,marcadet-poissonniers,5,9417.42,4.49,1.81,0.10,0.00,0.00,3.48,...,43.07,42.94,37.58,27.00,16.68,10.82,7.56,1883.48,941.74,104976.33
9,67,convention,5,7157.16,4.27,1.32,0.07,0.00,0.00,2.02,...,37.75,38.46,33.51,20.97,13.06,9.80,7.74,1431.43,715.72,104976.33


Tableau 'ligne numéro[13]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,54,basilique de saint-denis,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37332.82
1,76,plaisance,5,7378.74,4.13,1.31,0.03,0.00,0.00,5.80,...,39.72,38.27,32.66,22.15,12.53,8.94,7.01,1475.75,737.87,37332.82
2,155,malakoff-plateau de vanves,5,4647.68,2.93,1.02,0.09,0.00,0.00,3.69,...,39.62,41.65,31.21,18.21,9.74,7.87,5.92,929.54,464.77,37332.82
3,158,guy moquet,5,5378.58,5.08,2.09,0.06,0.00,0.00,2.90,...,37.81,36.78,33.77,20.83,12.24,9.86,8.41,1075.72,537.86,37332.82
4,187,champs-elysees-clemenceau,5,4301.32,15.47,5.89,0.09,0.00,0.02,3.31,...,62.33,67.18,52.39,31.77,25.55,24.32,23.10,860.26,430.13,37332.82
5,189,les agnettes,5,4123.32,2.65,0.77,0.04,0.00,0.00,9.27,...,34.57,31.37,25.58,17.10,10.05,7.16,5.00,824.66,412.33,37332.82
6,198,la fourche,5,3468.05,7.98,3.66,0.13,0.00,0.00,3.29,...,40.81,40.02,37.61,24.24,15.98,12.67,11.38,693.61,346.81,37332.82
7,290,varenne,5,1845.11,6.08,2.24,0.09,0.00,0.00,1.05,...,69.74,68.24,34.46,17.61,11.87,10.16,16.57,369.02,184.51,37332.82
8,35,chatillon-montrouge,5,11674.53,2.28,0.61,0.02,0.00,0.00,4.92,...,41.77,41.56,29.17,16.87,9.27,6.52,5.61,2334.91,1167.45,37332.82
9,39,mairie de saint-ouen,5,12017.74,7.75,2.71,0.19,0.01,0.01,11.46,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.55,1201.77,37332.82


Tableau 'ligne numéro[14]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,39,mairie de saint-ouen,5,12017.74,7.75,2.71,0.19,0.01,0.01,11.46,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.55,1201.77,47415.80
1,50,pont cardinet,5,9483.26,3.67,1.64,0.11,0.00,0.00,2.92,...,43.22,45.32,37.99,23.12,14.08,7.68,6.32,1896.65,948.33,47415.80
2,56,pyramides,5,8558.26,6.84,2.03,0.13,0.01,0.01,0.95,...,63.20,67.67,51.92,33.49,24.30,22.48,18.01,1711.65,855.83,47415.80
3,84,clichy saint-ouen,5,9117.84,2.59,1.24,0.18,0.01,0.00,4.59,...,54.71,54.96,36.11,20.36,12.13,6.13,4.06,1823.57,911.78,47415.80
4,6,bibliotheque,5,6088.00,6.46,1.35,0.06,0.00,0.10,5.02,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.60,608.80,47415.80
5,31,olympiades,5,12997.14,2.74,1.23,0.06,0.00,0.01,6.71,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.43,1299.71,47415.80
6,3,gare de lyon,5,66037.95,2.16,0.01,0.00,NaN,0.34,1.20,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.59,6603.79,47415.80
7,3,gare de lyon,5,66037.95,18.38,4.31,0.30,0.10,0.62,8.89,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.59,6603.79,47415.80
8,61,bercy,5,9023.95,NaN,NaN,NaN,NaN,NaN,6.98,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.79,902.39,47415.80
9,61,bercy,5,9023.95,10.59,2.39,0.22,0.02,0.30,3.90,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.79,902.39,47415.80


In [ ]:
from datetime import datetime

# Entrée de l'horaire sous forme de chaîne (par exemple, '14:30')
horaire_entree = input("Entrez l'horaire (format HH:MM) : ")

# Convertir la chaîne d'horaire en objet datetime pour travailler avec les heures
heure_entree = datetime.strptime(horaire_entree, '%H:%M').time()

# Définir les plages horaires
plages_horaires = [
    ('00:00', '01:00'), ('01:00', '02:00'), ('02:00', '03:00'), ('03:00', '04:00'),
    ('04:00', '05:00'), ('05:00', '06:00'), ('06:00', '07:00'), ('07:00', '08:00'),
    ('08:00', '09:00'), ('09:00', '10:00'), ('10:00', '11:00'), ('11:00', '12:00'),
    ('12:00', '13:00'), ('13:00', '14:00'), ('14:00', '15:00'), ('15:00', '16:00'),
    ('16:00', '17:00'), ('17:00', '18:00'), ('18:00', '19:00'), ('19:00', '20:00'),
    ('20:00', '21:00'), ('21:00', '22:00'), ('22:00', '23:00'), ('23:00', '00:00')
]

# Vérifier dans quelle plage horaire se trouve l'horaire entré
for plage in plages_horaires:
    debut, fin = map(lambda x: datetime.strptime(x, '%H:%M').time(), plage)
    if debut <= heure_entree < fin:
        print(f"L'horaire {horaire_entree} appartient à la plage horaire {plage[0]}-{plage[1]}")
        break

KeyboardInterrupt: Interrupted by user

In [ ]:
from datetime import datetime

# Entrée de la date et de l'horaire sous forme de chaînes
date_entree = input("Entrez la date (format YYYY-MM-DD) : ")
horaire_entree = input("Entrez l'horaire (format HH:MM) : ")

# Convertir la date et l'horaire en objets datetime
date_entree_dt = datetime.strptime(date_entree, '%Y-%m-%d').date()
heure_entree_dt = datetime.strptime(horaire_entree, '%H:%M').time()

# Combiner la date et l'horaire pour former un objet datetime complet
date_heure_entree_dt = datetime.combine(date_entree_dt, heure_entree_dt)

print(date_heure_entree_dt)

# Supposons que plages_horaires contient les plages horaires comme spécifié dans votre question

# Convertir les plages horaires en objets datetime.time
plages_horaires_dt = [(datetime.strptime(plage[0], '%H:%M').time(), datetime.strptime(plage[1], '%H:%M').time()) for plage in plages_horaires]

# Vérifier si la combinaison date + heure se trouve dans l'une des plages horaires
present = False

for plage_dt in plages_horaires_dt:
    if plage_dt[0] <= heure_entree_dt < plage_dt[1] and date_entree_dt == datetime.now().date():
        present = True
        break

# Renvoyer 'oui' si la combinaison date + heure est présente dans une plage horaire, sinon renvoyer '0'
if present:
    print('oui')
else:
    print('0')




2024-07-28 07:50:00
0
